In [62]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics import roc_auc_score, f1_score, mean_squared_error, make_scorer, accuracy_score
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
import matplotlib.pyplot as plt
import scipy

Был выбран датасет <a href='https://www.kaggle.com/mohansacharya/graduate-admissions'>Graduate Admission</a> , в котором описывается вероятность поступления в американские университеты на основе различных оценок, и факте проведения научных исследований

In [6]:
df = pd.read_csv('Admission_Predict_Ver1.1.csv')
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
495,496,332,108,5,4.5,4.0,9.02,1,0.87
496,497,337,117,5,5.0,5.0,9.87,1,0.96
497,498,330,120,5,4.5,5.0,9.56,1,0.93
498,499,312,103,4,4.0,5.0,8.43,0,0.73


Для того, чтобы решать задачу классификации, нам необходим параметр, который будет принимать значения 0 или 1. Возьмем факт того, что вероятность больше медианного значения. В качестве таргета для задачи регресси - возьмем просто вероятность поступления

In [9]:
median = np.median(df.iloc[:, -1])
median

0.72

In [13]:
df['target'] = df.iloc[:, -1] > median
df.target = df.target.values.astype(int)
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,target
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1
1,2,324,107,4,4.0,4.5,8.87,1,0.76,1
2,3,316,104,3,3.0,3.5,8.00,1,0.72,0
3,4,322,110,3,3.5,2.5,8.67,1,0.80,1
4,5,314,103,2,2.0,3.0,8.21,0,0.65,0
...,...,...,...,...,...,...,...,...,...,...
495,496,332,108,5,4.5,4.0,9.02,1,0.87,1
496,497,337,117,5,5.0,5.0,9.87,1,0.96,1
497,498,330,120,5,4.5,5.0,9.56,1,0.93,1
498,499,312,103,4,4.0,5.0,8.43,0,0.73,1


Разбиваем датасет на вектор фичей и таргетов для задачи классификации и регрессии. Для признаков отбрасываем первую, в которой находятся серийные номера поступающих - можем переобучиться на этих значениях и 2 последние колонки - это наш таргет

In [128]:
X = df.iloc[:, 1:-2].values
y_clf = df.iloc[:, -1].values
y_reg = df.iloc[:, -2].values

Разобьем наши данные на обучающую и тестовую выборку

In [142]:
X_train, X_test, y_clf_train, y_clf_test,  y_reg_train, y_reg_test = train_test_split(X, y_clf, y_reg)
X_test.shape[0] / X_train.shape[0]

0.3333333333333333

Тестовая выборка составляет треть от всех данных
<BR><BR>

Для того, чтобы выбрать лучший классификатор - произведем подбор гиперпараметров по сетке, с помошью кроссвалидации на четырех фолдах. В качестве метрики качества возьмем метрику f1

<img src='https://im0-tub-ru.yandex.net/i?id=3fdace876f7e30d11578717bc28f5d2d-l&n=13' width=200>

$$precision = \frac{TP}{TP + FP}$$

$$recall = \frac{TP}{TP + FN}$$

$$f1=\frac{1}{\frac{1}{precision} + \frac{1}{recall}}$$

In [130]:
params = {
    'criterion': ['gini', 'entropy'],
    'max_depth': np.linspace(3, 9, 4),
    'min_samples_split': np.linspace(0.2, 0.8, 4)
}
cv = KFold(4)
scorer = make_scorer(f1_score)
classifier = GridSearchCV(DecisionTreeClassifier(), params, scoring=scorer, cv=cv)
classifier.fit(X_train, y_clf_train)

GridSearchCV(cv=KFold(n_splits=4, random_state=None, shuffle=False),
             error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             pa

In [131]:
classifier.best_estimator_, classifier.best_score_

(DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                        max_depth=3.0, max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=0.4,
                        min_weight_fraction_leaf=0.0, presort='deprecated',
                        random_state=None, splitter='best'),
 0.8739067864243181)

Параметры, с которыми классификатор получил найбольший f1_score = 0.87:
<br>Критерий информативности - Джини
<br>Максимальная глубина дерева - 3
<br>Минимальная доля разделения в узле - 0.4

In [132]:
classifier = DecisionTreeClassifier(**classifier.best_params_)
classifier.fit(X_train, y_clf_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=3.0, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=0.4,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

Оценим качество работы нашего алгоритма на тестовых данных. Возмём 2 метрики: accuracy и f1

In [133]:
y_clf_pred = classifier.predict(X_test)
print("accuracy = {}\nf1 = {}".format(accuracy_score(y_clf_test, y_clf_pred), f1_score(y_clf_test, y_clf_pred)))

accuracy = 0.824
f1 = 0.819672131147541


Получили довольно неплохой результат. Точность близка к f1 - так получилось из-за того, что количество нулей и единиц примерно одинаково. 
<BR>Давайте посмотрим на самые важные признаки

In [135]:
columns = df.columns[1:-2]
importances = classifier.feature_importances_
ind = importances.argsort()[::-1]
for i in range(len(columns)):
    print("Feature: {}, importance: {}%".format(columns[ind[i]], 100 * importances[ind[i]]))


Feature: CGPA, importance: 86.04624666548507%
Feature: GRE Score, importance: 13.953753334514918%
Feature: Research, importance: 0.0%
Feature: LOR , importance: 0.0%
Feature: SOP, importance: 0.0%
Feature: University Rating, importance: 0.0%
Feature: TOEFL Score, importance: 0.0%


Похоже, самая важный признак - CGPA (среднее значение всех оценок по всем предметам). Именно на основе значений этого признака в большинстве случаев делались разделения на левые и правые подмножества в дереве. Второй по важности признак - GRE Score(Баллы за тест, который необходимо сдавать для поступления в аспирантуру, магистратуру или иной последипломный курс). Остальные признаки не использовались

<BR><BR>Перейдем к задачи регрессии. Опять проведем подбор гиперпараметров. Теперь за метрику качества будем брать MSE

$$
MSE = \frac{1}{n}\sum_{i = 1}^{n}{(y_i - a(x_{i})) ^ 2}
$$

<center>$y_i$ - истинное значение.
<br>$a(x_i)$ - предсказанное</center>

In [136]:
params = {
    'criterion': ["mse", "friedman_mse", "mae"],
    'max_depth': np.linspace(3, 9, 4),
    'min_samples_split': np.linspace(0.2, 0.8, 4)
}
cv = KFold(4)
scorer = make_scorer(mean_squared_error)
regressor = GridSearchCV(DecisionTreeRegressor(), params, scoring=scorer, cv=cv, verbose=1)
regressor.fit(X_train, y_reg_train)

Fitting 4 folds for each of 48 candidates, totalling 192 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 192 out of 192 | elapsed:    0.3s finished


GridSearchCV(cv=KFold(n_splits=4, random_state=None, shuffle=False),
             error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                             max_depth=None, max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=None,
                                             splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'criterion': ['mse', 'friedman_mse', 'mae'],
                      

In [137]:
regressor.best_estimator_, regressor.best_score_

(DecisionTreeRegressor(ccp_alpha=0.0, criterion='mae', max_depth=3.0,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=0.8,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best'),
 0.009790087365591399)

Лучшее полученное значение метрики MSE - 0.0098. 
<br><br><b>Лучшие гиперпараметры:</b>
<br>Критерий информативности - MAE
<br>Максимальная глубина дерева - 3
<br>Минимальная доля разделения в узле - 0.8

In [138]:
regressor = DecisionTreeRegressor(**regressor.best_params_)
regressor.fit(X_train, y_reg_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mae', max_depth=3.0,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=0.8,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

Для того, чтобы лучше оценить качество работы нашего алгоритма, возьмем еще метрику (относительную) - MAPE(Mean absolute percentage error)

$$
MAPE = \frac{1}{n}\sum_{i = 1}^{n}{\frac{|y_i - a(x_{i})|}{y_i}}
$$

In [139]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

y_reg_pred = regressor.predict(X_test)
mean_squared_error(y_reg_test, y_reg_pred), mean_absolute_percentage_error(y_reg_test, y_reg_pred)

(0.010473999999999999, 13.417077127397572)

MSE = 0.01, MAPE = 13.4% - хорошо. Давайте посмотрим на важность признаков

In [141]:
columns = df.columns[1:-2]
importances = regressor.feature_importances_
ind = importances.argsort()[::-1]
for i in range(len(columns)):
    print("Feature: {}, importance: {}%".format(columns[ind[i]], 100 * importances[ind[i]]))

Feature: CGPA, importance: 100.0%
Feature: Research, importance: 0.0%
Feature: LOR , importance: 0.0%
Feature: SOP, importance: 0.0%
Feature: University Rating, importance: 0.0%
Feature: TOEFL Score, importance: 0.0%
Feature: GRE Score, importance: 0.0%


Самым важным опять оказался признак CGPA. В задаче регресси остальные признаки не играли никакой роли